## 멀티 에이전트
멀티 에이전트 패턴 중 하나
- 도메인이 여러개 섞여 있을 때
- 각 도메인마다 도구가 많고 복잡할 때
- 하위 담당 에이전트와 사용자가 소통할 필요가 없을 때
- 단순 도구만 활용할 경우에는 사용 ❌

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain.chat_models import init_chat_model

model = init_chat_model('gpt-4.1-mini')

c:\Users\sscbr\AppData\Local\Programs\Python\Python314\Lib\site-packages\langchain_core\_api\deprecation.py:25: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


In [3]:
from langchain.tools import tool

@tool
def create_calendar_event(
    title: str,
    start_time: str,
    end_time: str,
    attendees: list[str],    # ['a@a.com', 'b@b.com'] 과 같은 list
    location: str = ''      # 위치가 없을 경우, 빈 문자열(Default Value 설정)
):
    '''캘린더 이벤트 생성'''
    return f'이벤트 생성 완료. {title} - {start_time} ~ {end_time}'

@tool
def send_email(
    to: list[str],
    subject: str,
    body: str
):
    '''이메일 발송'''
    return f'이메일 발송 완료. {to} - {subject}'

@tool
def get_available_time_slot(
    attendees: list[str],
    date: str,
    duration_minutes: int
):
    '''참가자들이 특정 날짜에 참여 가능한 시간 확인용'''

    return ['09:00', '14:00', '16:00']

In [4]:
from langchain.agents import create_agent
from datetime import datetime

CALENDAR_AGENT_PROMPT = (
    "You are a calendar scheduling assistant. "
    "Parse natural language scheduling requests (e.g., 'next Tuesday at 2pm') "
    "into proper ISO datetime formats. "
    "Use get_available_time_slots to check availability when needed. "
    "Use create_calendar_event to schedule events. "
    "Always confirm what was scheduled in your final response."
    f"NOW: {datetime.now()}"
)

calendar_agent = create_agent(
    model,
    tools=[create_calendar_event, get_available_time_slot],
    system_prompt=CALENDAR_AGENT_PROMPT
)

In [5]:
query = '다음주 화요일 2시에 1시간동안 팀 미팅을 잡아줘'

for step in calendar_agent.stream(
    {'messages': [{'role': 'user', 'content': query}]}
):
    for update in step.values():
        for message in update.get('messages', []):
            message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  get_available_time_slot (call_4gp8X0NLN93NDwPgprRhBc3f)
 Call ID: call_4gp8X0NLN93NDwPgprRhBc3f
  Args:
    attendees: ['팀']
    date: 2026-03-03
    duration_minutes: 60
================================= Tool Message =================================
Name: get_available_time_slot

["09:00", "14:00", "16:00"]
================================== Ai Message ==================================
Tool Calls:
  create_calendar_event (call_zA5ZqRCa2vsztPKxRLvvb6bv)
 Call ID: call_zA5ZqRCa2vsztPKxRLvvb6bv
  Args:
    title: 팀 미팅
    start_time: 2026-03-03T14:00:00
    end_time: 2026-03-03T15:00:00
    attendees: ['팀']
    location:
================================= Tool Message =================================
Name: create_calendar_event

이벤트 생성 완료. 팀 미팅 - 2026-03-03T14:00:00 ~ 2026-03-03T15:00:00
================================== Ai Message ==================================

다음주 화요일(3월 3일) 오후 2시부터 3

In [6]:
from datetime import datetime

EMAIL_AGENT_PROMPT = (
    "You are an email assistant. "
    "Compose professional emails based on natural language requests. "
    "Extract recipient information and craft appropriate subject lines and body text. "
    "Use send_email to send the message. "
    "Always confirm what was sent in your final response."
    f"NOW: {datetime.now()}"
)

email_agent = create_agent(
    model,
    tools=[send_email],
    system_prompt=EMAIL_AGENT_PROMPT,
)

In [7]:
query = '디자인 팀한테 내일 오전 10시에 있는 리뷰 리마인더 보내줘.'

for step in email_agent.stream(
    {'messages': [{'role': 'user', 'content': query}]}
):
    for update in step.values():
        for message in update.get('messages', []):
            message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  send_email (call_Ovk6a2P1kWrv9Gj6YOKD220s)
 Call ID: call_Ovk6a2P1kWrv9Gj6YOKD220s
  Args:
    to: ['design_team@example.com']
    subject: 리뷰 미팅 리마인더: 내일 오전 10시
    body: 안녕하세요, 디자인 팀 여러분.

내일 오전 10시에 예정된 리뷰 미팅이 있으니 잊지 말고 참석해 주시기 바랍니다.

감사합니다.
================================= Tool Message =================================
Name: send_email

이메일 발송 완료. ['design_team@example.com'] - 리뷰 미팅 리마인더: 내일 오전 10시
================================== Ai Message ==================================

내일 오전 10시에 있는 리뷰 미팅 리마인더를 디자인 팀에게 이메일로 보냈습니다. 더 필요하신 사항 있으면 알려주세요!


In [8]:
@tool
def schedule_event(request: str) -> str:
    """Schedule calendar events using natural language.

    Use this when the user wants to create, modify, or check calendar appointments.
    Handles date/time parsing, availability checking, and event creation.

    Input: Natural language scheduling request (e.g., 'meeting with design team
    next Tuesday at 2pm')
    """
    result = calendar_agent.invoke({
        "messages": [{"role": "user", "content": request}]
    })
    return result["messages"][-1].text       # 마지막 메세지만 관리자 에이전트에게 전달


@tool
def manage_email(request: str) -> str:
    """Send emails using natural language.

    Use this when the user wants to send notifications, reminders, or any email
    communication. Handles recipient extraction, subject generation, and email
    composition.

    Input: Natural language email request (e.g., 'send them a reminder about
    the meeting')
    """
    result = email_agent.invoke({
        "messages": [{"role": "user", "content": request}]
    })
    return result["messages"][-1].text       # 마지막 메세지만 관리자 에이전트에게 전달

In [9]:
SUPERVISOR_PROMPT='''너는 매우 유능한 개인 비서야.
너는 캘린더 이벤트를 조정하고, 이메일을 보낼 수 있어.
사용자의 요청을 분석하여 적절한 도구를 사용하고, 결과를 종합해야해.
요청이 여러가지 액션을 취해야 하면, 순서를 잘 짜서 각종 도구들을 여러번 호출해.
'''

supervisor_agent = create_agent(
    model,
    tools=[schedule_event, manage_email],
    system_prompt=SUPERVISOR_PROMPT
)

query = '디자인팀엔 alice, bob, charlie가 있어. 다음주 화요일 오후 2시부터 1시간동안 디자인 팀 미팅을 잡고, 참여자들에게 메일 보내줘.'

for step in supervisor_agent.stream(
    {'messages': [{'role': 'user', 'content': query}]}
):
    for update in step.values():
        for message in update.get('messages', []):
            message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  schedule_event (call_YyY0ZCnmbRrtqWBHA7rkYzcv)
 Call ID: call_YyY0ZCnmbRrtqWBHA7rkYzcv
  Args:
    request: 디자인팀 미팅 next Tuesday at 2pm for 1 hour
  manage_email (call_OsOBq9FEbh0nm0MGjwrZgsqs)
 Call ID: call_OsOBq9FEbh0nm0MGjwrZgsqs
  Args:
    request: send an email to alice, bob, and charlie about the design team meeting scheduled for next Tuesday at 2pm for 1 hour
================================= Tool Message =================================
Name: manage_email

The email about the design team meeting scheduled for next Tuesday at 2 PM for 1 hour has been sent to Alice, Bob, and Charlie.
================================= Tool Message =================================
Name: schedule_event

디자인팀 미팅을 2026년 3월 3일 화요일 오후 2시부터 3시까지 1시간 동안 예약했습니다.
================================== Ai Message ==================================

디자인팀 미팅을 다음주 화요일 오후 2시부터 1시간 동안으로 예약 완료했습니다. 그리고 Alice, Bob, Charli

In [10]:
# 실습용(신규 테이블 추가 + Agent)
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit

import os

DB_URI = os.environ.get('DB_URI')

db = SQLDatabase.from_uri(DB_URI)
toolkit = SQLDatabaseToolkit(db=db, llm=model)

dialect = db.dialect
top_k = 5

SQL_AGENT_PROMPT = f"""
You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run,
then look at the results of the query and return the answer. Unless the user
specifies a specific number of examples they wish to obtain, always limit your
query to at most {top_k} results.

You can order the results by a relevant column to return the most interesting
examples in the database. Never query for all the columns from a specific table,
only ask for the relevant columns given the question.

You MUST double check your query before executing it. If you get an error while
executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the
database.

To start you should ALWAYS look at the tables in the database to see what you
can query. Do NOT skip this step.

Then you should query the schema of the most relevant tables.
"""

db_agent = create_agent(
    model, 
    toolkit.get_tools(), 
    system_prompt=SQL_AGENT_PROMPT,
)
query = '백엔드 팀에 있는 사람들의 이메일주소만 알려줘'

for step in db_agent.stream(
    {'messages': [{'role': 'user', 'content': query}]}
):
    for update in step.values():
        for message in update.get('messages', []):
            message.pretty_print()
@tool
def schedule_event(request: str) -> str:
    """Schedule calendar events using natural language.

    Use this when the user wants to create, modify, or check calendar appointments.
    Handles date/time parsing, availability checking, and event creation.

    Input: Natural language scheduling request (e.g., 'meeting with design team
    next Tuesday at 2pm')
    """
    result = calendar_agent.invoke({
        "messages": [{"role": "user", "content": request}]
    })
    return result["messages"][-1].text


@tool
def manage_email(request: str) -> str:
    """Send emails using natural language.

    Use this when the user wants to send notifications, reminders, or any email
    communication. Handles recipient extraction, subject generation, and email
    composition.

    Input: Natural language email request (e.g., 'send them a reminder about
    the meeting')
    """
    result = email_agent.invoke({
        "messages": [{"role": "user", "content": request}]
    })
    return result["messages"][-1].text


@tool
def query_db(request: str) -> str:
    """Query Database using natural language.

    Team info and employee info are in Database.
    Use this when you need to query DB.

    Input: Natural language query request (e.g., 'members in data team.')
    """
    result = db_agent.invoke({
        "messages": [{"role": "user", "content": request}]
    })
    return result["messages"][-1].text
from langgraph.checkpoint.memory import InMemorySaver

SUPERVISOR_PROMPT = '''너는 매우 유능한 개인 비서야.
너는 아래와 같은 일을 할 수있어.

1. DB에서 팀, 멤버 정보 쿼리.
2. 캘린더 이벤트를 조정
3. 이메일을 전송 (DB에서 메일 주소 참조 필요)

사용자의 요청을 분석하여, 적절한 도구를 사용하고, 결과를 종합해야해.
요청이 여러가지 액션을 취해야 하면, 순서를 잘 짜서 각종 도구들을 여러번 호출해.
'''
supervisor_model = init_chat_model('gpt-4.1-mini')

supervisor_agent = create_agent(
    supervisor_model,
    tools=[schedule_event, manage_email, query_db],
    system_prompt=SUPERVISOR_PROMPT,
    checkpointer=InMemorySaver()
)
query = '아까 한거 오후 4시로 바꾸자'

config = {'configurable': {'thread_id': '12345d6'}}


for step in supervisor_agent.stream(
    {'messages': [{'role': 'user', 'content': query}]},
    config=config,
):
    for update in step.values():
        for message in update.get('messages', []):
            message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_9SLrdKDghkk1d9mM6GTrJkf9)
 Call ID: call_9SLrdKDghkk1d9mM6GTrJkf9
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

courses, customers, dt_demo, employees, inventory, lotto_draws, members, sales, sample, students, students_courses, teams, test_n8n_chat_histories
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_5CjEaHSj02zsTJCC2fNyfM0W)
 Call ID: call_5CjEaHSj02zsTJCC2fNyfM0W
  Args:
    table_names: employees, teams
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE employees (
	id SERIAL NOT NULL, 
	name VARCHAR(100) NOT NULL, 
	email VARCHAR(150) NOT NULL, 
	team_id INTEGER NOT NULL, 
	CONSTRAINT employees_pkey PRIMARY KEY (id), 
	CONSTRAINT fk_team FOREIGN KEY(team_id) REF